 - ## Use RNN_LSTM to train

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data
tf.set_random_seed(1)   # set random seed
import heapq

C:\Users\user\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


 - get train data

In [2]:
# item_num_loc_day_temp = pd.read_csv('item_num_rate_loc_day_temp.csv')
item_num_loc_day_temp = pd.read_csv('item_num_loc_day_temp.csv')

In [3]:
item_num_loc_day_temp = item_num_loc_day_temp.drop(['Unnamed: 0', 'Unnamed: 0.1'], axis=1)

In [4]:
item_num_loc_day_temp.head()

,City,ItemId,Quantity,month,rain,temp,year
0,南投縣,1162471.0,2.0,519,0.0,13.05,2015
1,南投縣,1119696.0,2.0,603,0.0,13.35,2015
2,南投縣,1209835.0,3.0,613,0.0,14.15,2015
3,南投縣,1194488.0,2.0,614,0.0,14.55,2015
4,南投縣,1175590.0,3.0,618,0.0,14.25,2015


In [5]:
dict_region = {"北部":['臺北','台北','新北','基隆','宜蘭','桃園','新竹'],
               "中部":['苗栗','臺中','台中','彰化','南投','雲林'],
               "南部":['嘉義','臺南','台南','高雄','屏東','澎湖'],
               "東部":['花蓮','台東','臺東']
              }

In [6]:
def get_keys(d, seri_state):
    
    result = list()
    for state in seri_state:
        locate_flag = 0
        for key,value in d.items():
            for a in value:            
                if a in state:
                    result.append(key)
                    locate_flag = 1
                    break
            if locate_flag == 1:
                break
        if locate_flag == 0:
            result.append('not_define')
    return result
    
get_keys(dict_region, ['桃園'])

['北部']

In [7]:
# 時間轉換
timee = []
for i in range(len(item_num_loc_day_temp)):
    mon = str(item_num_loc_day_temp['month'][i])
    yea = str(item_num_loc_day_temp['year'][i])
    if len(mon) == 4:
        a_date = yea+mon[0:2]+mon[-2:]
    if len(mon) == 3:
        a_date = yea+'0'+mon[0]+mon[-2:]
    timee.append(a_date)

In [8]:
item_num_loc_day_temp['Date'] = pd.Series(timee)
item_num_loc_day_temp['new_DateId'] = pd.to_datetime(item_num_loc_day_temp['Date'].astype(str),format='%Y%m%d')

In [9]:
item_num_loc_day_temp['Area'] = get_keys(dict_region, item_num_loc_day_temp['City'])
item_num_loc_day_temp_1 = item_num_loc_day_temp

In [10]:
# 3718  9193
item_num_loc_day_temp = item_num_loc_day_temp_1[item_num_loc_day_temp_1['Area'] == '南部']

In [11]:
# get rnn_train_data  ==> temp , rain
year = ['5','6','7','8']
month = ['01','02','03','04','05','06','07','08','09','10','11','12']
day = ['01','07','13','19','25','31']
day_ = ['28','30']

In [12]:
week_rain = []
week_temp = []
for q in range(len(year)):
    y = year[q]
    for r in range(len(month)):
        m = month[r]        
        for s in range((len(day)-1)):
            d = day[s]
            d1 = day[s+1]
            if d1 == '31':
                if m == '02':
                    d1 = day_[0]
                if (m == '04') or (m == '06') or (m == '09') or (m == '11'):
                    d1 = day_[1]            
            aa = item_num_loc_day_temp[(item_num_loc_day_temp['new_DateId'] > '201'+y+'-'+m+'-'+d) & (item_num_loc_day_temp['new_DateId'] < '201'+y+'-'+m+'-'+d1)]
            week_rain.append(aa['rain'].mean())
            week_temp.append(aa['temp'].mean())

In [13]:
# get  rnn_train_data  ==> tf_ItemId
week_temp = [x for x in week_temp if str(x) != 'nan']
week_rain = [x for x in week_rain if str(x) != 'nan']
len(week_rain),len(week_temp)

(177, 177)

In [14]:
all_item = list(((item_num_loc_day_temp_1).groupby(by='ItemId')['Quantity'].sum()).index)
len(all_item)

1774

In [15]:
# Create Item_sum_zero_vector
all_item_zero = []
for i in range(len(all_item)):
    all_item_zero.append(0)
all_item = pd.Series(all_item)
all_item_zero = pd.Series(all_item_zero)
tf_all_item_zero = {'item':all_item,'sum':all_item_zero}
tf_all_item_zero = np.array(pd.DataFrame(tf_all_item_zero))   

In [16]:
week_item_tf = []
for q in range(len(year)):
    y = year[q]
    for r in range(len(month)):
        m = month[r]        
        for s in range((len(day)-1)):
            d = day[s]
            d1 = day[s+1]
            if d1 == '31':
                if m == '02':
                    d1 = day_[0]
                if (m == '04') or (m == '06') or (m == '09') or (m == '11'):
                    d1 = day_[1]            
            aa = item_num_loc_day_temp[(item_num_loc_day_temp['new_DateId'] > '201'+y+'-'+m+'-'+d) & (item_num_loc_day_temp['new_DateId'] < '201'+y+'-'+m+'-'+d1)]
            if len(aa)>0:
                all_item_zero = []
                for i in range(len(all_item)):
                    all_item_zero.append(0)
                all_item = pd.Series(all_item)
                all_item_zero = pd.Series(all_item_zero)
                tf_all_item_zero = {'item':all_item,'sum':all_item_zero}
                tf_all_item_zero = np.array(pd.DataFrame(tf_all_item_zero)) 
                n = tf_all_item_zero
                yu = ((aa).groupby(by='ItemId')['Quantity'].sum())
                for p in range(len(yu)):
                    ty_item_name = list(yu.index)[p]
                    tf_item_num = np.array(yu)[p]                    
                    ru = [i for i,item in enumerate(n) if ty_item_name in item]
                    n[ru[0]][1] = int(tf_item_num)
                week_item_tf.append(n)

In [17]:
week_rain.append(week_rain[0])
week_temp.append(week_temp[0])
week_item_tf.append(week_item_tf[0])

In [18]:
(np.array(week_item_tf)).shape,len(week_temp)

((178, 1774, 2), 178)

In [19]:
aa = {}
week_item_sum = []
for i in range(len(week_item_tf)):
    for j in range(len((week_item_tf[0]))):
        week_item_sum.append(week_item_tf[i][j][1])
    aa.update({('item'+str(i)): pd.Series(np.array(week_item_sum))})    
    week_item_sum = []
week_item_sum_df = pd.DataFrame(aa)

In [20]:
week_item_sum = []
week_item_sum_1 = []
week_item_sum = np.vstack([np.array(week_item_sum_df['item4']),np.array(week_item_sum_df['item5'])])
week_item_sum_1 = np.vstack([np.array(week_item_sum_df['item91']),np.array(week_item_sum_df['item92'])])
for i in range(6,91):
    week_item_sum = np.vstack([week_item_sum,np.array(week_item_sum_df['item'+str(i)])])
for i in range(93,178):
    week_item_sum_1 = np.vstack([week_item_sum_1,np.array(week_item_sum_df['item'+str(i)])])

In [21]:
len(week_item_tf),len(week_rain),len(week_temp)

(178, 178, 178)

In [22]:
# Get Top 5 Item
week_item_sum_five = week_item_sum
week_item_sum_five_1 = week_item_sum_1

for i in range(len(week_item_sum)):
    mydata = heapq.nlargest(10, enumerate(week_item_sum_five[i]), key=lambda x:x[1])
    mydata_1 = heapq.nlargest(10, enumerate(week_item_sum_five_1[i]), key=lambda x:x[1])
    
    indexes, values = zip(*mydata)
    indexes_1, values_1 = zip(*mydata_1)
    
    week_item_sum_five[i] = week_item_sum_five[i] - week_item_sum_five[i]
    week_item_sum_five_1[i] = week_item_sum_five_1[i] - week_item_sum_five_1[i]
    for j in range(10):
        week_item_sum_five[i][indexes[j]] = 1 #values[j]
        week_item_sum_five_1[i][indexes_1[j]] = 1 #values_1[j]

 - ## Start RNN_LSTM

In [23]:
# hyperparameters
lr = 0.001                  # learning rate
epoch = 251                   # train step 上限
batch_size = 87            
n_inputs = 2                # data_feature  [雨量、氣溫 ]
n_steps = 7                 # time steps
n_hidden_units = 128        # neurons in hidden layer
n_classes = 1774            # Item_ID num 
rnn_len = 4                 # 4 區 = 4 cell

In [24]:
# create x y placeholder
x = [tf.placeholder(tf.float32, shape=[None, n_inputs], name='input') for i in range(rnn_len)]
y = tf.placeholder(tf.float32, [None, n_classes])

 - Functions of weight and bias

In [25]:
weights = {
    # shape (2, 128)
    'in': tf.Variable(tf.random_normal([n_inputs, n_hidden_units])),
    # shape (128, 1774)
    'out': tf.Variable(tf.random_normal([n_hidden_units, n_classes]))
}
biases = {
    # shape (128, )
    'in': tf.Variable(tf.constant(0.1, shape=[n_hidden_units, ])),
    # shape (1774, )
    'out': tf.Variable(tf.constant(0.1, shape=[n_classes, ]))
}

 - Encoder inputs and outputs

In [26]:
# LSTM cell
LSTM = tf.contrib.rnn.BasicLSTMCell(n_hidden_units)

# encoder input
x = [tf.placeholder(tf.float32, shape=[None, n_inputs], name='input') for i in range(rnn_len)] # create 4 placeholder [features]

rnn_input = [tf.nn.relu(tf.add(tf.matmul(x[i], weights['in']), biases['in'])) for i in range(rnn_len)] #  [hidden layer_1] 、relu = 避免太線性

# encoder output
with tf.variable_scope('enc'):
    rnn_output, _ = tf.contrib.rnn.static_rnn(cell=LSTM,
                                              inputs=rnn_input,
                                              dtype=tf.float32)

In [27]:
rnn_output = rnn_output[-1]

In [28]:
rnn_output

<tf.Tensor 'enc/rnn/basic_lstm_cell/Mul_11:0' shape=(?, 128) dtype=float32>

In [29]:
_y_hat = tf.add(tf.matmul(rnn_output, weights['out']), biases['out']) #  [hidden layer_1] 、relu = 避免太線性
y_hat = tf.nn.softmax(_y_hat)

In [30]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=_y_hat, labels=y))
cost

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.



<tf.Tensor 'Mean:0' shape=() dtype=float32>

In [31]:
train_op = tf.train.AdamOptimizer(lr).minimize(cost)

In [32]:
batch_x1 = [np.transpose(np.vstack([week_temp[0:87],week_rain[0:87]])),
            np.transpose(np.vstack([week_temp[1:88],week_rain[1:88]])),
            np.transpose(np.vstack([week_temp[2:89],week_rain[2:89]])),
            np.transpose(np.vstack([week_temp[3:90],week_rain[3:90]]))]

batch_x2 = [np.transpose(np.vstack([week_temp[87:174],week_rain[87:174]])),
            np.transpose(np.vstack([week_temp[88:175],week_rain[88:175]])),
            np.transpose(np.vstack([week_temp[89:176],week_rain[89:176]])),
            np.transpose(np.vstack([week_temp[90:177],week_rain[90:177]]))]

batch_y1 =  list(week_item_sum)                              # 87*1774  4->91, 91->178

batch_y2 =  list(week_item_sum_1)

In [33]:
(np.array(batch_x1)).shape,(np.array(batch_x2)).shape,(np.array(batch_y1)).shape,(np.array(batch_y2)).shape

((4, 87, 2), (4, 87, 2), (87, 1774), (87, 1774))

In [34]:
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

In [35]:
for i in range(epoch):
    sess.run([train_op], feed_dict={
        x[0]: batch_x1[0],
        x[1]: batch_x1[1],
        x[2]: batch_x1[2],
        x[3]: batch_x1[3],        
        y: batch_y1
    })    
    sess.run([train_op], feed_dict={
        x[0]: batch_x2[0],
        x[1]: batch_x2[1],
        x[2]: batch_x2[2],
        x[3]: batch_x2[3],        
        y: batch_y2
    })
    if i % 25 == 0:
        test = sess.run(y_hat, feed_dict={
        x[0]: batch_x2[0],
        x[1]: batch_x2[1],
        x[2]: batch_x2[2],
        x[3]: batch_x2[3],            
        })
        test        #v.s. batch_y2
        # Get Top 5 Item
        te = test
        for j in range(len(te)):
            mydata = heapq.nlargest(10, enumerate(te[j]), key=lambda x:x[1])
            indexes, values = zip(*mydata)
            te[j] = te[j] - te[j]
            for k in range(10):
                te[j][indexes[k]] = 1 #values[j]
#                 print('Recommender ItemID = ',all_item[indexes[k]])
        accuracy = []
        for h in range(len(te)):
            accuracy.append(20-sum(abs(te[h] - week_item_sum_five_1[h])))
        print('Train',i,' = ',(np.mean(accuracy))*5 ,'%')

Train 0  =  0.3448275862068966 %
Train 25  =  14.25287356321839 %
Train 50  =  14.137931034482758 %
Train 75  =  14.827586206896552 %
Train 100  =  18.27586206896552 %
Train 125  =  16.666666666666668 %
Train 150  =  17.126436781609197 %
Train 175  =  19.42528735632184 %
Train 200  =  18.96551724137931 %
Train 225  =  18.850574712643677 %
Train 250  =  17.816091954022987 %


In [36]:
prdeict_south_loc=[]
for j in range(len(te)):
    mydata = heapq.nlargest(10, enumerate(te[j]), key=lambda x:x[1])
    indexes, values = zip(*mydata)    
    for k in range(10):
        if k % 10 == 0:
            print('Recommender ItemID','step',j)
        print(all_item[indexes[k]])
        prdeict_south_loc.append(all_item[indexes[k]])

Recommender ItemID step 0
1077959.0
1077986.0
1077999.0
1078006.0
1119355.0
1910545.0
1922989.0
1959878.0
2294453.0
2467800.0
Recommender ItemID step 1
1077959.0
1077986.0
1077999.0
1078006.0
1119355.0
1910545.0
1922989.0
1959878.0
2294453.0
2467800.0
Recommender ItemID step 2
1077959.0
1077986.0
1077999.0
1078006.0
1119355.0
1910545.0
1922989.0
1959878.0
2294453.0
2467800.0
Recommender ItemID step 3
1077959.0
1077986.0
1077999.0
1078006.0
1119355.0
1910545.0
1922989.0
1959878.0
2294453.0
2467800.0
Recommender ItemID step 4
1077959.0
1077986.0
1077999.0
1078006.0
1119355.0
1910545.0
1922989.0
1959878.0
2294453.0
2467800.0
Recommender ItemID step 5
1077959.0
1077986.0
1077999.0
1078006.0
1119355.0
1910545.0
1922989.0
1959878.0
2294453.0
2467800.0
Recommender ItemID step 6
1077959.0
1077986.0
1077999.0
1078006.0
1119355.0
1910545.0
1922989.0
1959878.0
2294453.0
2467800.0
Recommender ItemID step 7
1077959.0
1077986.0
1077999.0
1078006.0
1119355.0
1910545.0
1922989.0
1959878.0
2294453.0
24

In [37]:
thefile = open('prdeict_south_loc.txt', 'w')
for item in prdeict_south_loc:
    thefile.write("%s\n" % item)
# prdeict_all_loc.to_csv('prdeict_all_loc.csv')